#**Movie Recommendation system**

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import difflib
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import files
files.upload()

In [25]:
movies = pd.read_csv('/content/movies.csv')
movies.head(3)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


In [30]:
#shape of the datasets
movies.shape

(4803, 24)

In [31]:
#information about the dataset
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [33]:
#columns in the dataset
movies.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [34]:
#data types the dataset
movies.dtypes

index                     int64
budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
director                 object
dtype: object

In [35]:
#statistics for numerical value
movies.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [36]:
#statistics for object value
movies.describe(include='object')

,genres,homepage,keywords,original_language,original_title,overview,production_companies,production_countries,release_date,spoken_languages,status,tagline,title,cast,crew,director
count,4775,1712,4391,4803,4803,4800,4803,4803,4802,4803,4803,3959,4803,4760,4803,4773
unique,1168,1691,4219,37,4801,4800,3697,469,3280,544,3,3944,4800,4741,4776,2349
top,Drama,http://www.missionimpossible.com/,independent film,en,Out of the Blue,"In the 22nd century, a paraplegic Marine is di...",[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2006-01-01,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Based on a true story.,The Host,William Shatner Leonard Nimoy DeForest Kelley ...,[],Steven Spielberg
freq,370,4,55,4505,2,1,351,2977,10,3171,4795,3,2,6,28,27


In [40]:
#checking for missing values
#statistics for numerical value
movies.isnull().sum()


index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [42]:
columns_with_missing_values = movies.loc[:, movies.isnull().sum() > 0]
columns_with_missing_values.head()

,genres,homepage,keywords,overview,release_date,runtime,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",2009-12-10,162.0,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",2007-05-19,169.0,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,2015-10-26,148.0,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,2012-07-16,165.0,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,http://movies.disney.com/john-carter,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",2012-03-07,132.0,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [43]:
#feature selection
selected_feature = ['genres','keywords','tagline','cast','director']
new_movies = movies[selected_feature] 
new_movies.head()

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [45]:
new_movies.isnull().sum()

genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64

In [47]:
#replacing missing values with empty string
new_movies = new_movies.fillna('')

In [48]:
new_movies.isnull().sum()

genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

In [58]:
def combine_features(movies):
    """
    Combine multiple features of a movie into a single string.

    Args:
    movies (pandas.DataFrame): A dataframe with columns 'genres', 'keywords', 'tagline', 'cast', and 'director'.

    Returns:
    pandas.Series: A series of strings, where each string is the concatenation of the five features.

    """
    # Define a list of features to be combined
    features = ['genres', 'keywords', 'tagline', 'cast', 'director']

    # Initialize an empty string to store the combined features
    combined_features = ""

    # Iterate through each feature and concatenate its values with a space separator
    for feature in features:
        # Fill any missing values with an empty string and convert to string type
        combined_features += movies[feature].fillna('').astype(str) + ' '

    # Strip any leading/trailing whitespace and return the combined string as a pandas Series
    return combined_features.str.strip()

# Call the function on the new_movies dataframe
combined_features = combine_features(new_movies)
combined_features.head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
Name: genres, dtype: object

In [ ]:

def get_feature_vector(data):
    """
    Convert a list of text data into a sparse matrix of TF-IDF vectors.

    Args:
    data (list): A list of strings to be vectorized.

    Returns:
    scipy.sparse.csr_matrix: A sparse matrix of shape (n_samples, n_features) representing the TF-IDF vectors of the input data.

    """

    # Define a TfidfVectorizer object to convert text to TF-IDF vectors
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer on the input data and transform it into a sparse matrix
    feature_vector = vectorizer.fit_transform(data)

    return feature_vector

# Call the function on the combined features
feature_vector = get_feature_vector(combined_features)
print(feature_vector[10])


In [ ]:

def get_cosine_similarity_matrix(feature_vector):
    """
    Compute the cosine similarity matrix of a feature vector.

    Args:
    feature_vector (scipy.sparse.csr_matrix): A sparse matrix of shape (n_samples, n_features) representing the feature vectors of a dataset.

    Returns:
    numpy.ndarray: A square matrix of shape (n_samples, n_samples) representing the pairwise cosine similarity scores between the samples.

    """

    # Compute the cosine similarity matrix using the feature vectors
    similarity_matrix = cosine_similarity(feature_vector)

    return similarity_matrix

# Call the function on the feature vector
similarity = get_cosine_similarity_matrix(feature_vector)
similarity[0]


In [209]:

title2 = [re.sub(r'[\s-]','',movies) for movies in movies['title']]
movies['title2'] = title2
movies['title2'] = movies['title2'].str.lower()

In [188]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,title2
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron,Avatar
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski,PiratesoftheCaribbean:AtWorld'sEnd
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes,Spectre
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan,TheDarkKnightRises
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton,JohnCarter


In [217]:

def find_similar_movies(movies, similarity):
    """
    This function takes in a Pandas DataFrame of movie titles and a cosine similarity matrix of the features of the movies
    and returns a list of similar movies based on the user's input movie name.

    Args:
    movies (Pandas DataFrame): DataFrame of movie titles and their features.
    similarity (numpy array): cosine similarity matrix of the features of the movies.

    Returns:
    Pandas DataFrame: DataFrame containing the top 20 most similar movies and their corresponding similarity score.
    """
    #getting input from the user
    movie_name = input('Enter Your favourite movie name: ')
    # preprocess user input and movie names
    movie_name = re.sub(r'\W+', '', movie_name.lower())

    #creating a list with all the movies names given in the dataset
    movie_list = movies['title2'].tolist()
    #finding the close match for the movie name given by the user
    get_close_match = difflib.get_close_matches(movie_name,movie_list)
    if len(get_close_match) == 0:

      # handle empty list
      print('No close matches found')
    else:
      close_match = get_close_match[0]

    if not get_close_match:
        print('Sorry, no close matches found for the given movie name.')
        return None
    else:
        print()
        # Print the close matches found
        print('\nClose matches:')
        for i, match in enumerate(get_close_match):
          print(f"{i+1}. {match}")
          print()
        selected_index_str = input("Enter the number corresponding to the movie you meant: ")
        try:
          selected_index = int(selected_index_str)
        except ValueError:
          print(f"Invalid input. Please enter a number between 1 and {len(get_close_match)}")

        close_match = get_close_match[0]
        accurate_close_match = get_close_match[selected_index-1] 
        print('\nFirst match is:', close_match)

        print('\nAccurate match is:',accurate_close_match)

        #find the index of the movie with title
        index_of_the_movie = movies[movies.title2 == accurate_close_match]['index'].values[0]
        print('\nindex of the movie:', index_of_the_movie)

        #getting a list of a similar movies
        similarity_score = list(enumerate(similarity[index_of_the_movie]))

        #sorting the movies based on the similarity score
        sorted_similar_movies = sorted(similarity_score, key= lambda x:x[1],reverse=True )

        #print the name of similar movies 
        print('Movies suggested for you:')
        titles = []
        for movie in sorted_similar_movies[:21]:
            index = movie[0]
            score = round(movie[1],3)
            title = movies[movies.index == index]['title'].values[0]
            titles.append({
                'Movie name':title,
                'Movie Index':index,
                'Similarity Score':score
            })
        movie_rank = pd.DataFrame(titles)
        return movie_rank


In [220]:
find_similar_movies(movies,similarity)

Enter Your favourite movie name: X-Men


Close matches:
1. xmen

2. amen.

3. taxman

Enter the number corresponding to the movie you meant: 1

First match is: xmen

Accurate match is: xmen

index of the movie: 511
Movies suggested for you:


,Movie name,Movie Index,Similarity Score
0,X-Men,511,1.000
1,X2,203,0.802
2,X-Men: The Last Stand,33,0.768
3,X-Men: Days of Future Past,46,0.534
4,X-Men: Apocalypse,64,0.420
5,The Wolverine,232,0.317
6,Avengers: Age of Ultron,7,0.305
7,Ant-Man,182,0.290
8,The Avengers,16,0.289
9,X-Men Origins: Wolverine,122,0.276
